In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
from scipy.stats import shapiro
pd.options.display.max_columns= None
pd.options.display.max_rows= None

#to surpress the notation'e'
pd.options.display.float_format='{:.6f}'.format

from sklearn.model_selection import train_test_split
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.stattools import durbin_watson
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso,Ridge,ElasticNet,SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score , mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

# CHENNAI

## BASIC MODEL BUILDING

In [180]:
df_c = pd.read_csv('Chennai.csv')

In [181]:
df_c = df_c.replace(9,np.nan)

In [182]:
df_c.shape

(5014, 40)

In [183]:
df_c = df_c.dropna(axis=0)

In [184]:
df_num = df_c.select_dtypes(include=np.number)

In [185]:
vif = pd.DataFrame()

In [186]:
vif['VIF'] = [VIF(df_num.values,i) for i in range(df_num.shape[1])]
vif['Features'] = df_num.columns
vif.sort_values(['VIF'],ascending=False)

,VIF,Features
1,25.278593,Area
2,15.938799,No. of Bedrooms
35,14.361889,DiningTable
5,10.965466,Gymnasium
38,10.247442,Refrigerator
28,7.467229,Children'splayarea
34,6.343860,TV
15,6.217481,ClubHouse
6,6.103066,SwimmingPool
8,5.627480,JoggingTrack


## Feature Engineering

### Feature Selection

In [187]:
from sklearn.model_selection import train_test_split

In [188]:
df_c.set_index('Location', inplace=True)

In [189]:
ss = StandardScaler()

In [190]:
df_c['Area']=ss.fit_transform(np.array(df_c['Area']).reshape(-1,1))

In [191]:
df_c['Price']=ss.fit_transform(np.array(df_c['Price']).reshape(-1,1))

In [192]:
x = df_c.drop('Price',1)
y = df_c['Price']

In [193]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=100)

In [194]:
lr = LinearRegression()

In [195]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [196]:
bwd = sfs(estimator = lr,k_features='best',forward=False,cv=5,scoring='r2')

In [197]:
bwd_mod = bwd.fit(x_train,y_train)

In [198]:
bwd_names = bwd_mod.k_feature_names_
bwd_feat = list(bwd_names)
bwd_feat

['Area',
 'No. of Bedrooms',
 'LandscapedGardens',
 'JoggingTrack',
 'IndoorGames',
 'ShoppingMall',
 'ClubHouse',
 'School',
 'CarParking',
 'StaffQuarter',
 'Cafeteria',
 'MultipurposeRoom',
 'AC',
 'Wifi',
 'BED',
 'Microwave',
 'GolfCourse',
 'Sofa']

# Assumptions

### Regression

In [199]:
x = df_c[['Area',
 'No. of Bedrooms',
 'LandscapedGardens',
 'JoggingTrack',
 'IndoorGames',
 'ShoppingMall',
 'ClubHouse',
 'School',
 'CarParking',
 'StaffQuarter',
 'Cafeteria',
 'MultipurposeRoom',
 'AC',
 'Wifi',
 'BED',
 'Microwave',
 'GolfCourse',
 'Sofa']]
y = df_c['Price']

In [200]:
xc = sm.add_constant(x)
ols_c = sm.OLS(y,xc).fit()
ols_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     255.0
Date:                Thu, 09 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:57:41   Log-Likelihood:                -2002.5
No. Observations:                2233   AIC:                             4039.
Df Residuals:                    2216   BIC:                             4136.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.5533      0.069      8.029      0.000       0.418       0.688
Area                  0.9073      0.020     44.664      0.000       0.867       0.947
No. of Bedrooms      -0.2110      0.029     -7.371      0.000      -0.267      -0.155
LandscapedGardens     0.3222      0.044      7.255      0.000       0.235       0.409
JoggingTrack         -0.2088      0.046     -4.508      0.000      -0.300      -0.118
IndoorGames          -0.1468      0.042     -3.528      0.000      -0.228      -0.065
ShoppingMall         -0.1643      0.097     -1.691      0.091      -0.355       0.026
ClubHouse            -0.0604      0.036     -1.687      0.092      -0.131       0.010
School               -0.2531      0.077     -3.299      0.001      -0.404      -0.103
CarParking            0.1582      0.040      3.976      0.000       0.080       0.236
StaffQuarter          0.0310      0.066      0.468      0.640      -0.099       0.161
Cafeteria            -0.3070      0.050     -6.157      0.000      -0.405      -0.209
MultipurposeRoom     -0.1850      0.043     -4.333      0.000      -0.269      -0.101
AC                   -0.0273      0.145     -0.188      0.851      -0.312       0.257
Wifi               7.063e-17   3.14e-16      0.225      0.822   -5.45e-16    6.86e-16
BED                   0.2005      0.116      1.736      0.083      -0.026       0.427
Microwave             0.1371      0.181      0.756      0.450      -0.219       0.493
GolfCourse                 0          0        nan        nan           0           0
Sofa                  0.7299      0.174      4.202      0.000       0.389       1.071
==============================================================================
Omnibus:                     1741.581   Durbin-Watson:                   1.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           172706.835
Skew:                           2.986   Prob(JB):                         0.00
Kurtosis:                      45.668   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [201]:
print('The Accuracy of the above CHENNAI DATASET using OLS method iS:',ols_c.rsquared)

The Accuracy of the above CHENNAI DATASET using OLS method iS: 0.6480715765319841


# FINAL MODEL BUILDING

# Hyperparameter tuning

In [202]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error

In [203]:
from sklearn.model_selection import GridSearchCV

In [204]:
def build_model(instance):
    model=instance.fit(xtrain,y_train)
    print("Train R2:",model.score(xtrain,y_train))
    print("Test R2:",model.score(xtest,y_test))
    print("Train RMSE:",np.sqrt(mean_squared_error(y_train,model.predict(xtrain))))
    print("Test RMSE:",np.sqrt(mean_squared_error(y_test,model.predict(xtest))))

### LASSO

In [257]:
param = {'alpha':[0.01,0.02,0.1,0.9,0.85,0.001,0.0001,0.0003,0.005]}
l = Lasso(normalize=True)

lassocv = GridSearchCV(l,param,cv=5,scoring='r2')
lassocv.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=Lasso(normalize=True),
             param_grid={'alpha': [0.01, 0.02, 0.1, 0.9, 0.85, 0.001, 0.0001,
                                   0.0003, 0.005]},
             scoring='r2')

In [258]:
lassocv.best_params_

{'alpha': 0.0003}

In [259]:
xtrain=x_train
ytrain=y_train
xtest=x_test
ytest=y_test

In [261]:
lasso001 = Lasso(alpha=0.0003,normalize=True)

build_model(lasso001)

Train R2: 0.6662160018014702
Test R2: 0.5768579078304616
Train RMSE: 0.6164853232784105
Test RMSE: 0.5343424056193413


# ENET

In [265]:
from sklearn.model_selection import GridSearchCV

param = {'alpha':[0.01,0.02,0.001,0.0005,0.0001,0.003,0.0005,0.00005],
        'l1_ratio':[0.3,0.4,0.6,0.8,0.9,1,2,3]}
enet = ElasticNet(normalize=True)

enetcv = GridSearchCV(enet,param,cv=5,scoring='r2')
enetcv.fit(xtrain,ytrain)
enetcv.best_params_

{'alpha': 0.0001, 'l1_ratio': 0.8}

In [266]:
enetmod = ElasticNet(alpha= 0.0001,l1_ratio= 0.8)

In [267]:
build_model(enetmod)

Train R2: 0.6747959904814139
Test R2: 0.5475853297155916
Train RMSE: 0.6085102964983002
Test RMSE: 0.5525160093835366


### RIDGE

In [212]:
rd = Ridge()

In [213]:
param = {'alpha':[0.01,0.02,0.1,0.001]}

In [214]:
rd.get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'random_state', 'solver', 'tol'])

In [233]:
from sklearn.model_selection import GridSearchCV

param=[{'alpha':[0.001,0.005,0.01,0.1,0.2,0.5,0.8,1,2,3,4,5,6,7,8,9,10,11,12,20,22,23,25]}]         
         
rdmod = GridSearchCV(rd,param,cv=5,scoring='r2')
rdmod.fit(xtrain,ytrain)
rdmod.best_params_

{'alpha': 22}

In [235]:
rdmodel = Ridge(alpha= 23)

In [236]:
build_model(rdmodel)

Train R2: 0.6713320094591615
Test R2: 0.5677109460060467
Train RMSE: 0.611742551714796
Test RMSE: 0.5400869034967332


In [218]:
#cross val score

In [219]:
cross_val_score(estimator=rd,X=xtrain,y=y_train,cv=5 )

array([0.64759351, 0.67180155, 0.67545856, 0.56807121, 0.57691915])

# BANGALORE

## BASIC MODEL BUILDING

In [42]:
df_b = pd.read_csv('Bangalore.csv')

In [43]:
df_b = df_b.replace(9,np.nan)

In [44]:
df_b.shape

(6207, 40)

In [45]:
df_b = df_b.dropna(axis=0)

In [46]:
df_num = df_b.select_dtypes(include=np.number)

In [47]:
vif = pd.DataFrame()

In [48]:
vif['VIF'] = [VIF(df_num.values,i) for i in range(df_num.shape[1])]
vif['Features'] = df_num.columns
vif.sort_values(['VIF'],ascending=True)

,VIF,Features
33,1.458881,GolfCourse
4,1.603694,MaintenanceStaff
21,1.745728,Cafeteria
14,1.957069,ATM
3,2.406167,Resale
20,3.115271,StaffQuarter
11,3.139284,ShoppingMall
25,3.511907,Gasconnection
31,3.815339,VaastuCompliant
22,4.117803,MultipurposeRoom


## Feature Engineering

### Feature Selection

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
df_b.set_index('Location', inplace=True)

In [51]:
df_b['Area']=ss.fit_transform(np.array(df_b['Area']).reshape(-1,1))

df_b['Price']=ss.fit_transform(np.array(df_b['Price']).reshape(-1,1))

In [52]:
x = df_b.drop('Price',1)
y = df_b['Price']

In [53]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=100)

In [54]:
lr = LinearRegression()

In [55]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [56]:
bwd = sfs(estimator = lr,k_features='best',forward=False,cv=5,scoring='r2')

In [57]:
bwd_mod = bwd.fit(x_train,y_train)

In [58]:
bwd_names = bwd_mod.k_feature_names_
bwd_feat = list(bwd_names)
bwd_feat

['Area',
 'No. of Bedrooms',
 'MaintenanceStaff',
 'SwimmingPool',
 'LandscapedGardens',
 'JoggingTrack',
 'RainWaterHarvesting',
 'ShoppingMall',
 'Intercom',
 'ATM',
 'ClubHouse',
 'School',
 'StaffQuarter',
 'Cafeteria',
 'WashingMachine',
 "Children'splayarea",
 'BED',
 'VaastuCompliant',
 'GolfCourse',
 'TV',
 'DiningTable']

# Assumptions

### Regression

In [59]:
x = df_b[['Area',
 'No. of Bedrooms',
 'MaintenanceStaff',
 'SwimmingPool',
 'LandscapedGardens',
 'JoggingTrack',
 'RainWaterHarvesting',
 'ShoppingMall',
 'Intercom',
 'ATM',
 'ClubHouse',
 'School',
 'StaffQuarter',
 'Cafeteria',
 'WashingMachine',
 "Children'splayarea",
 'BED',
 'VaastuCompliant',
 'GolfCourse',
 'TV',
 'DiningTable']]
y = df_b['Price']

In [60]:
xc = sm.add_constant(x)
ols_c = sm.OLS(y,xc).fit()
ols_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.856
Model:                            OLS   Adj. R-squared:                  0.854
Method:                 Least Squares   F-statistic:                     602.2
Date:                Thu, 09 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:34:45   Log-Likelihood:                -880.62
No. Observations:                1951   AIC:                             1801.
Df Residuals:                    1931   BIC:                             1913.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7240      0.054     13.375      0.000       0.618       0.830
Area                    1.0467      0.012     83.743      0.000       1.022       1.071
No. of Bedrooms        -0.3169      0.018    -17.304      0.000      -0.353      -0.281
MaintenanceStaff        0.0537      0.030      1.783      0.075      -0.005       0.113
SwimmingPool            0.1547      0.033      4.650      0.000       0.089       0.220
LandscapedGardens      -0.2581      0.026     -9.975      0.000      -0.309      -0.207
JoggingTrack            0.0277      0.027      1.018      0.309      -0.026       0.081
RainWaterHarvesting     0.1721      0.024      7.264      0.000       0.126       0.219
ShoppingMall            0.1983      0.040      4.997      0.000       0.120       0.276
Intercom                0.0605      0.023      2.591      0.010       0.015       0.106
ATM                    -0.1040      0.033     -3.182      0.001      -0.168      -0.040
ClubHouse               0.0353      0.027      1.288      0.198      -0.018       0.089
School                 -0.2853      0.039     -7.317      0.000      -0.362      -0.209
StaffQuarter            0.0834      0.030      2.780      0.005       0.025       0.142
Cafeteria              -0.0616      0.024     -2.516      0.012      -0.110      -0.014
WashingMachine          0.1518      0.074      2.043      0.041       0.006       0.298
Children'splayarea     -0.0130      0.035     -0.370      0.712      -0.082       0.056
BED                    -0.4742      0.222     -2.135      0.033      -0.910      -0.039
VaastuCompliant        -0.1131      0.023     -4.868      0.000      -0.159      -0.068
GolfCourse             -0.1750      0.052     -3.369      0.001      -0.277      -0.073
TV                      0.1518      0.074      2.043      0.041       0.006       0.298
DiningTable             0.1518      0.074      2.043      0.041       0.006       0.298
==============================================================================
Omnibus:                     1735.244   Durbin-Watson:                   1.327
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           154701.166
Skew:                           3.754   Prob(JB):                         0.00
Kurtosis:                      45.973   Cond. No.                     2.14e+32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.3e-61. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [61]:
print('The Accuracy of the above BANGALORE DATASET using OLS method iS:',ols_c.rsquared)

The Accuracy of the above BANGALORE DATASET using OLS method iS: 0.8555960347836125


# FINAL MODEL BUILDING

# Hyperparameter tuning

In [62]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error

In [63]:
from sklearn.model_selection import GridSearchCV

In [64]:
def build_model(instance):
    model=instance.fit(xtrain,y_train)
    print("Train R2:",model.score(xtrain,y_train))
    print("Test R2:",model.score(xtest,y_test))
    print("Train RMSE:",np.sqrt(mean_squared_error(y_train,model.predict(xtrain))))
    print("Test RMSE:",np.sqrt(mean_squared_error(y_test,model.predict(xtest))))

### LASSO

In [65]:
param = {'alpha':[0.01,0.02,0.1,0.95,0.90,0.001,0.0001,0.0006,0.0004]}
l = Lasso(normalize=True)

lassocv = GridSearchCV(l,param,cv=5,scoring='r2')
lassocv.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=Lasso(normalize=True),
             param_grid={'alpha': [0.01, 0.02, 0.1, 0.95, 0.9, 0.001, 0.0001,
                                   0.0006, 0.0004]},
             scoring='r2')

In [66]:
lassocv.best_params_

{'alpha': 0.0001}

In [67]:
xtrain=x_train
ytrain=y_train
xtest=x_test
ytest=y_test

In [68]:
lasso001 = Lasso(alpha=0.0001,normalize=True)

build_model(lasso001)

Train R2: 0.8665776293547037
Test R2: 0.8209964960459237
Train RMSE: 0.38944827385913866
Test RMSE: 0.34858654688714236


# ENET

In [69]:
from sklearn.model_selection import GridSearchCV

param = {'alpha':[0.01,0.02,0.001,0.0005,0.0004,0.00005,0.00003],
        'l1_ratio':[0.3,0.4,0.6,0.8,0.9]}
enet = ElasticNet(normalize=True)

enetcv = GridSearchCV(enet,param,cv=5,scoring='r2')
enetcv.fit(xtrain,ytrain)
enetcv.best_params_

{'alpha': 5e-05, 'l1_ratio': 0.3}

In [70]:
enetmod = ElasticNet(alpha= 5e-05,l1_ratio= 0.3)

In [71]:
build_model(enetmod)

Train R2: 0.8677941002609901
Test R2: 0.817516740104482
Train RMSE: 0.3876688222715952
Test RMSE: 0.3519584297104483


### RIDGE

In [72]:
rd = Ridge()

In [73]:
param = {'alpha':[0.01,0.02,0.1,0.001]}

In [74]:
from sklearn.model_selection import GridSearchCV

param = {'alpha':[0.01,0.099,0.001,0.0005,0.0001,0.00005,0.0999]}
         
         
rdmod = GridSearchCV(rd,param,cv=5,scoring='r2')
rdmod.fit(xtrain,ytrain)
rdmod.best_params_

{'alpha': 0.0999}

In [75]:
rdmodel2 = Ridge(alpha= 0.0999)

In [76]:
build_model(rdmodel2)

Train R2: 0.867794547508173
Test R2: 0.8175243874661765
Train RMSE: 0.3876681665371097
Test RMSE: 0.3519510548377766


# HYDERABAD

## BASIC MODEL BUILDING

In [77]:
df_h = pd.read_csv('Hyderabad.csv')

In [78]:
df_h = df_h.replace(9,np.nan)

In [79]:
df_h.shape

(2518, 40)

In [80]:
df_h= df_h.dropna(axis=0)

In [81]:
df_num = df_h.select_dtypes(include=np.number)

In [82]:
vif = pd.DataFrame()

In [83]:
vif['VIF'] = [VIF(df_num.values,i) for i in range(df_num.shape[1])]
vif['Features'] = df_num.columns
vif.sort_values(['VIF'],ascending=True)

,VIF,Features
27,1.451814,Wifi
3,1.665369,Resale
37,1.745113,Wardrobe
33,1.769352,GolfCourse
4,1.810477,MaintenanceStaff
25,1.909397,Gasconnection
20,1.987606,StaffQuarter
30,1.995914,BED
21,2.220652,Cafeteria
26,2.261703,AC


## Feature Engineering

### Feature Selection

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
df_h.set_index('Location', inplace=True)

In [86]:
df_h['Area']=ss.fit_transform(np.array(df_h['Area']).reshape(-1,1))

df_h['Price']=ss.fit_transform(np.array(df_h['Price']).reshape(-1,1))

In [87]:
x = df_h.drop('Price',1)
y = df_h['Price']

In [88]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=100)

In [89]:
lr = LinearRegression()

In [90]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [91]:
bwd = sfs(estimator = lr,k_features='best',forward=False,cv=5,scoring='r2')

In [92]:
bwd_mod = bwd.fit(x_train,y_train)

In [93]:
bwd_names = bwd_mod.k_feature_names_
bwd_feat = list(bwd_names)
bwd_feat

['Area',
 'No. of Bedrooms',
 'Resale',
 'MaintenanceStaff',
 'Gymnasium',
 'SwimmingPool',
 'RainWaterHarvesting',
 'IndoorGames',
 'SportsFacility',
 'ATM',
 'ClubHouse',
 '24X7Security',
 'PowerBackup',
 'CarParking',
 'Cafeteria',
 'WashingMachine',
 'Gasconnection',
 'BED',
 'Microwave',
 'GolfCourse',
 'TV',
 'Refrigerator']

# Assumptions

### Regression

In [94]:
x = df_h[['Area',
 'No. of Bedrooms',
 'Resale',
 'MaintenanceStaff',
 'Gymnasium',
 'SwimmingPool',
 'RainWaterHarvesting',
 'IndoorGames',
 'SportsFacility',
 'ATM',
 'ClubHouse',
 '24X7Security',
 'PowerBackup',
 'CarParking',
 'Cafeteria',
 'WashingMachine',
 'Gasconnection',
 'BED',
 'Microwave',
 'GolfCourse',
 'TV',
 'Refrigerator']]
y = df_h['Price']

In [95]:
xc = sm.add_constant(x)
ols_c = sm.OLS(y,xc).fit()
ols_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     308.9
Date:                Thu, 09 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:34:52   Log-Likelihood:                -1823.1
No. Observations:                2434   AIC:                             3692.
Df Residuals:                    2411   BIC:                             3826.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.3950      0.069      5.712      0.000       0.259       0.531
Area                    0.9190      0.018     51.674      0.000       0.884       0.954
No. of Bedrooms        -0.1849      0.025     -7.267      0.000      -0.235      -0.135
Resale                  0.1519      0.027      5.573      0.000       0.098       0.205
MaintenanceStaff       -0.1288      0.038     -3.365      0.001      -0.204      -0.054
Gymnasium              -0.0974      0.052     -1.891      0.059      -0.198       0.004
SwimmingPool            0.1002      0.051      1.952      0.051      -0.000       0.201
RainWaterHarvesting    -0.0646      0.039     -1.673      0.094      -0.140       0.011
IndoorGames             0.1366      0.041      3.337      0.001       0.056       0.217
SportsFacility         -0.0748      0.037     -2.026      0.043      -0.147      -0.002
ATM                     0.0827      0.039      2.115      0.035       0.006       0.159
ClubHouse               0.0862      0.047      1.842      0.066      -0.006       0.178
24X7Security           -0.0970      0.045     -2.151      0.032      -0.185      -0.009
PowerBackup             0.0438      0.029      1.500      0.134      -0.013       0.101
CarParking              0.0501      0.039      1.298      0.194      -0.026       0.126
Cafeteria               0.1021      0.042      2.411      0.016       0.019       0.185
WashingMachine          0.0313      0.127      0.246      0.806      -0.219       0.281
Gasconnection           0.0838      0.047      1.782      0.075      -0.008       0.176
BED                     0.0717      0.056      1.270      0.204      -0.039       0.182
Microwave              -0.1887      0.078     -2.433      0.015      -0.341      -0.037
GolfCourse             -0.1659      0.070     -2.356      0.019      -0.304      -0.028
TV                      0.2334      0.103      2.263      0.024       0.031       0.436
Refrigerator            0.0114      0.122      0.093      0.926      -0.227       0.250
==============================================================================
Omnibus:                     4514.143   Durbin-Watson:                   1.802
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         18509917.452
Skew:                          13.071   Prob(JB):                         0.00
Kurtosis:                     429.416   Cond. No.                         48.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [96]:
print('The Accuracy of the above HYDERABAD DATASET using OLS method iS:',ols_c.rsquared)

The Accuracy of the above HYDERABAD DATASET using OLS method iS: 0.7381092977870087


# FINAL MODEL BUILDING

# Hyperparameter tuning

In [97]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error

In [98]:
from sklearn.model_selection import GridSearchCV

In [99]:
def build_model(instance):
    model=instance.fit(xtrain,y_train)
    print("Train R2:",model.score(xtrain,y_train))
    print("Test R2:",model.score(xtest,y_test))
    print("Train RMSE:",np.sqrt(mean_squared_error(y_train,model.predict(xtrain))))
    print("Test RMSE:",np.sqrt(mean_squared_error(y_test,model.predict(xtest))))

### LASSO

In [100]:
param = {'alpha':[0.01,0.02,0.1,0.88,0.9,0.95,0.99,0.00001,0.00002,0.00005]}
l = Lasso(normalize=True)

lassocv = GridSearchCV(l,param,cv=5,scoring='r2')
lassocv.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=Lasso(normalize=True),
             param_grid={'alpha': [0.01, 0.02, 0.1, 0.88, 0.9, 0.95, 0.99,
                                   1e-05, 2e-05, 5e-05]},
             scoring='r2')

In [101]:
lassocv.best_params_

{'alpha': 2e-05}

In [102]:
xtrain=x_train
ytrain=y_train
xtest=x_test
ytest=y_test

In [103]:
lasso001 = Lasso(alpha=0.00002,normalize=True)

build_model(lasso001)

Train R2: 0.833529904022327
Test R2: 0.6103371170857217
Train RMSE: 0.36663452840825417
Test RMSE: 0.7497093400314876


# ENET

In [104]:
from sklearn.model_selection import GridSearchCV

param = {'alpha':[0.01,0.02,0.001,0.0005,0.0006,0.00005],
        'l1_ratio':[0.3,0.4,0.6,0.8,0.9]}
enet = ElasticNet(normalize=True)

enetcv = GridSearchCV(enet,param,cv=5,scoring='r2')
enetcv.fit(xtrain,ytrain)
enetcv.best_params_

{'alpha': 5e-05, 'l1_ratio': 0.9}

In [105]:
enetmod = ElasticNet(alpha= 5e-05,l1_ratio= 0.9)

In [106]:
build_model(enetmod)

Train R2: 0.8336487808445368
Test R2: 0.6101489410314677
Train RMSE: 0.3665035975912112
Test RMSE: 0.7498903430516213


### RIDGE

In [107]:
rd = Ridge()

In [108]:
param = {'alpha':[0.01,0.02,0.1,0.001]}

In [109]:
from sklearn.model_selection import GridSearchCV

param = {'alpha':[0.01,0.02,0.098,0.099,0.01,0.0005,0.0001,0.00005]}
         
         
rdmod = GridSearchCV(rd,param,cv=5,scoring='r2')
rdmod.fit(xtrain,ytrain)
rdmod.best_params_

{'alpha': 0.099}

In [110]:
rdmodel3 = Ridge(alpha= 0.099)

In [111]:
build_model(rdmodel3)

Train R2: 0.8336530033741816
Test R2: 0.6100726320303733
Train RMSE: 0.366498946040912
Test RMSE: 0.7499637308002965


# DELHI

## BASIC MODEL BUILDING

In [112]:
df_d = pd.read_csv('Delhi.csv')

In [113]:
df_d = df_d.replace(9,np.nan)

In [114]:
df_d.shape

(4998, 40)

In [115]:
df_d= df_d.dropna(axis=0)

In [116]:
df_num = df_d.select_dtypes(include=np.number)

In [117]:
vif = pd.DataFrame()

In [118]:
vif['VIF'] = [VIF(df_num.values,i) for i in range(df_num.shape[1])]
vif['Features'] = df_num.columns
vif.sort_values(['VIF'],ascending=False)

,VIF,Features
1,28.127606,Area
16,19.834316,School
23,19.110647,Hospital
2,15.920423,No. of Bedrooms
35,9.913117,DiningTable
36,8.930239,Sofa
24,8.874244,WashingMachine
0,8.231133,Price
38,7.229285,Refrigerator
8,5.972926,JoggingTrack


## Feature Engineering

### Feature Selection

In [119]:
from sklearn.model_selection import train_test_split

In [120]:
df_d.set_index('Location', inplace=True)

In [121]:
df_d['Area']=ss.fit_transform(np.array(df_d['Area']).reshape(-1,1))

df_d['Price']=ss.fit_transform(np.array(df_d['Price']).reshape(-1,1))

In [122]:
x = df_d.drop('Price',1)
y = df_d['Price']

In [123]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=100)

In [124]:
lr = LinearRegression()

In [125]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [126]:
bwd = sfs(estimator = lr,k_features='best',forward=False,cv=5,scoring='r2')

In [127]:
bwd_mod = bwd.fit(x_train,y_train)

In [128]:
bwd_names = bwd_mod.k_feature_names_
bwd_feat = list(bwd_names)
bwd_feat

['Area',
 'Resale',
 'JoggingTrack',
 'RainWaterHarvesting',
 'IndoorGames',
 'Intercom',
 'School',
 '24X7Security',
 'CarParking',
 'StaffQuarter',
 'Cafeteria',
 'MultipurposeRoom',
 'Hospital',
 'WashingMachine',
 'Gasconnection',
 'AC',
 'LiftAvailable',
 'BED',
 'VaastuCompliant',
 'Microwave',
 'DiningTable',
 'Refrigerator']

# Assumptions

### Regression

In [129]:
x = df_d[['Area',
 'Resale',
 'JoggingTrack',
 'RainWaterHarvesting',
 'IndoorGames',
 'Intercom',
 'School',
 '24X7Security',
 'CarParking',
 'StaffQuarter',
 'Cafeteria',
 'MultipurposeRoom',
 'Hospital',
 'WashingMachine',
 'Gasconnection',
 'AC',
 'LiftAvailable',
 'BED',
 'VaastuCompliant',
 'Microwave',
 'DiningTable',
 'Refrigerator']]
y = df_d['Price']

In [130]:
xc = sm.add_constant(x)
ols_c = sm.OLS(y,xc).fit()
ols_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                     234.1
Date:                Thu, 09 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:34:59   Log-Likelihood:                -1557.9
No. Observations:                2002   AIC:                             3162.
Df Residuals:                    1979   BIC:                             3291.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0240      0.033     -0.736      0.462      -0.088       0.040
Area                    0.7542      0.016     48.055      0.000       0.723       0.785
Resale                  0.1307      0.033      3.921      0.000       0.065       0.196
JoggingTrack            0.0748      0.127      0.589      0.556      -0.174       0.324
RainWaterHarvesting     0.4695      0.065      7.254      0.000       0.343       0.596
IndoorGames            -0.0291      0.156     -0.186      0.852      -0.336       0.277
Intercom               -0.0832      0.030     -2.798      0.005      -0.142      -0.025
School                  1.3138      0.422      3.117      0.002       0.487       2.140
24X7Security           -0.4493      0.092     -4.880      0.000      -0.630      -0.269
CarParking              0.0877      0.036      2.468      0.014       0.018       0.157
StaffQuarter           -0.1984      0.130     -1.529      0.126      -0.453       0.056
Cafeteria              -0.0157      0.141     -0.111      0.911      -0.292       0.261
MultipurposeRoom       -0.2129      0.108     -1.969      0.049      -0.425      -0.001
Hospital               -1.1477      0.443     -2.593      0.010      -2.016      -0.280
WashingMachine         -0.2899      0.188     -1.539      0.124      -0.659       0.080
Gasconnection           0.1032      0.034      3.011      0.003       0.036       0.170
AC                      0.4484      0.062      7.287      0.000       0.328       0.569
LiftAvailable          -0.1941      0.027     -7.100      0.000      -0.248      -0.141
BED                    -0.4807      0.095     -5.062      0.000      -0.667      -0.295
VaastuCompliant        -0.1033      0.039     -2.656      0.008      -0.180      -0.027
Microwave               0.4448      0.096      4.650      0.000       0.257       0.632
DiningTable            -0.8778      0.194     -4.521      0.000      -1.259      -0.497
Refrigerator            1.0341      0.152      6.818      0.000       0.737       1.332
==============================================================================
Omnibus:                     2839.895   Durbin-Watson:                   1.768
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1295462.229
Skew:                           7.967   Prob(JB):                         0.00
Kurtosis:                     126.597   Cond. No.                         84.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [131]:
print('The Accuracy of the above DELHI DATASET using OLS method iS:',ols_c.rsquared)

The Accuracy of the above DELHI DATASET using OLS method iS: 0.7223947647980735


# FINAL MODEL BUILDING

# Hyperparameter tuning

In [132]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error

In [133]:
from sklearn.model_selection import GridSearchCV

In [134]:
def build_model(instance):
    model=instance.fit(xtrain,y_train)
    print("Train R2:",model.score(xtrain,y_train))
    print("Test R2:",model.score(xtest,y_test))
    print("Train RMSE:",np.sqrt(mean_squared_error(y_train,model.predict(xtrain))))
    print("Test RMSE:",np.sqrt(mean_squared_error(y_test,model.predict(xtest))))

### LASSO

In [135]:
param = {'alpha':[0.01,0.02,0.1,0.96,0.99,0.001]}
l = Lasso(normalize=True)

lassocv = GridSearchCV(l,param,cv=5,scoring='r2')
lassocv.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=Lasso(normalize=True),
             param_grid={'alpha': [0.01, 0.02, 0.1, 0.96, 0.99, 0.001]},
             scoring='r2')

In [136]:
lassocv.best_params_

{'alpha': 0.001}

In [137]:
xtrain=x_train
ytrain=y_train
xtest=x_test
ytest=y_test

In [138]:
lasso001 = Lasso(alpha=0.001,normalize=True)

build_model(lasso001)

Train R2: 0.6878482271928981
Test R2: 0.7068281650278037
Train RMSE: 0.5790676747703121
Test RMSE: 0.49238060150866736


# ENET

In [139]:
from sklearn.model_selection import GridSearchCV

param = {'alpha':[0.01,0.02,0.001,0.0005,0.0001,0.00005],
        'l1_ratio':[0.3,0.4,0.6,0.8,0.9]}
enet = ElasticNet(normalize=True)

enetcv = GridSearchCV(enet,param,cv=5,scoring='r2')
enetcv.fit(xtrain,ytrain)
enetcv.best_params_

{'alpha': 5e-05, 'l1_ratio': 0.3}

In [140]:
enetmod = ElasticNet(alpha= 5e-05,l1_ratio= 0.3)

In [141]:
build_model(enetmod)

Train R2: 0.728879713721305
Test R2: 0.699475886640454
Train RMSE: 0.5396689435682667
Test RMSE: 0.4985164275789127


### RIDGE

In [142]:
rd = Ridge()

In [143]:
param = {'alpha':[0.01,0.02,0.1,0.001]}

In [144]:
from sklearn.model_selection import GridSearchCV

param = {'alpha':[0.01,0.02,0.099,0.085,0.001,0.0005,0.0001,0.00005]}
         
         
rdmod = GridSearchCV(rd,param,cv=5,scoring='r2')
rdmod.fit(xtrain,ytrain)
rdmod.best_params_

{'alpha': 0.099}

In [145]:
rdmodel = Ridge(alpha= 0.099)

In [146]:
build_model(rdmodel)

Train R2: 0.7288664380386505
Test R2: 0.6997208755808311
Train RMSE: 0.5396821561279571
Test RMSE: 0.4983131894583352
